In [1]:
from scraper import WebScraper

In [2]:
keywords = ["Sperrkonto"]  # Stichwörter für die Suche
foldername = "Results_from_scraper"

In [ ]:
url = "https://www.berlin.de/einwanderung/service/faq/"
scraper = WebScraper(keywords)
scraper.scrape(url)
# Option 1: als csv Speichern
scraper.save_results_to_file(filename = f"berlinde_{'_'.join(map(str, keywords))}", folder = foldername, filetype = "csv")

# Option 2: als JSON speichern
scraper.save_results_to_file(filename = f"berlinde_{'_'.join(map(str, keywords))}", folder = foldername, filetype = "json")

In [ ]:
second_url = "https://www.studying-in-germany.org/de/sperrkonto-auslaendische-studenten/"
scraper = WebScraper(keywords)
scraper.scrape(second_url)
scraper.save_results_to_file(filename= f"studying-in-germanyorg_{'_'.join(map(str, keywords))}", folder = foldername, filetype="json")